In [1]:
#!pip install torch torchvision torchaudio
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader
import numpy as np
from pathlib import Path
import torch.nn.functional as F

home = Path('/data')
train_dir = home / 'datasets/imagenet_full_size/061417/train'
test_dir = home / 'datasets/imagenet_full_size/061417/val'

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

# Apply transform_test for deterministic features
trainset = datasets.ImageFolder(train_dir, transform=transform)
testset = datasets.ImageFolder(test_dir, transform=transform)

# Sample 10% of trainset
subset_size = int(0.1 * len(trainset))
indices = np.random.choice(len(trainset), subset_size, replace=False)
train_subset = Subset(trainset, indices)

train_loader = DataLoader(train_subset, batch_size=64, shuffle=False, num_workers=4)
test_loader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=4)

In [2]:

import sklearn
import torch
from train import ResNet18

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Recreate model architecture
n_classes = 1000  # or whatever your original number was
lda_args = None   # or use the same lda_args you trained with

checkpoint = torch.load("models/deeplda_best.pth", map_location=device)

# Rebuild model
embedding_model = ResNet18(num_classes=n_classes, lda_args=lda_args)
embedding_model.load_state_dict(checkpoint['state_dict'])
embedding_model.to(device)
embedding_model.eval()




ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_

In [5]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score


# Initialize the LDA model
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')  # Optional: you can tweak parameters

# Fit the LDA model
lda.fit(X_train.cpu().numpy(), y_train.cpu().numpy())

# Make predictions on the test set
y_pred = lda.predict(X_test.cpu().numpy())

# Calculate accuracy
acc = accuracy_score(y_test.cpu().numpy(), y_pred)
print(f"Test Accuracy: {acc * 100:.2f}%")


Test Accuracy: 8.20%


In [4]:
def extract_embeddings(dataloader, model, device):
    model.eval()
    embeddings, labels = [], []
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)
            feats = model._forward_features(x)  # Direct embeddings
            feats = F.normalize(feats, p=2, dim=1)  # L2 normalization
            embeddings.append(feats)
            labels.append(y)
    return torch.cat(embeddings), torch.cat(labels)

X_train, y_train = extract_embeddings(train_loader, embedding_model, device)
X_test, y_test = extract_embeddings(test_loader, embedding_model, device)

/root/miniconda3/envs/myenv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/root/miniconda3/envs/myenv/lib/python3.10/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [8]:
# import torch
# import torch.nn.functional as F
# from sklearn.metrics import accuracy_score

# # --- Compute class centroids (normalized) ---
# def compute_normalized_centroids_torch(X, y):
#     classes = torch.unique(y)
#     centroids = []
#     for cls in classes:
#         class_feats = X[y == cls]
#         mean = class_feats.mean(dim=0, keepdim=True)
#         mean = F.normalize(mean, p=2, dim=1)
#         centroids.append(mean)
#     return torch.cat(centroids, dim=0), classes

# # --- Predict based on cosine similarity ---
# def predict_cosine_torch(X, centroids, classes):
#     X = F.normalize(X, p=2, dim=1)
#     sims = torch.matmul(X, centroids.T)
#     pred_indices = sims.argmax(dim=1)
#     return classes[pred_indices]

# # --- Main ---
# # Make sure these are torch tensors on the right device
# X_train_torch = X_train.to(device)
# y_train_torch = y_train.to(device)
# X_test_torch = X_test.to(device)
# y_test_torch = y_test.to(device)

# centroids, class_labels = compute_normalized_centroids_torch(X_train_torch, y_train_torch)
# y_pred_torch = predict_cosine_torch(X_test_torch, centroids, class_labels)

# # Convert to CPU and NumPy for scoring
# acc = accuracy_score(y_test.cpu().numpy(), y_pred_torch.cpu().numpy())
# print(f"Centroid Cosine Classifier Accuracy: {acc * 100:.2f}%")

model_state = embedding_model.state_dict()
checkpoint_state = checkpoint['state_dict']

model_keys = set(model_state.keys())
checkpoint_keys = set(checkpoint_state.keys())




for name in model_state:
    if name in checkpoint_state:
        if not torch.allclose(model_state[name], checkpoint_state[name], atol=1e-6):
            print(f"Value mismatch in {name}")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

class LinearClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.linear = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.linear(x)

# --- Config ---
batch_size = 4096
lr = 1e-2
epochs = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- Prepare data ---
train_ds = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

test_ds = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_ds, batch_size=batch_size)

# --- Model, Loss, Optimizer ---
model = LinearClassifier(X_train.shape[1], int(y_train.max().item()) + 1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# --- Training ---
for epoch in range(epochs):
    model.train()
    correct, total = 0, 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        loss = criterion(out, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        preds = out.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

    acc = correct / total * 100
    print(f"Epoch {epoch+1}: Train Accuracy = {acc:.2f}%")

# --- Evaluation ---
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        preds = model(xb).argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

test_acc = correct / total * 100
print(f"Test Accuracy = {test_acc:.2f}%")
